In [1]:
import pandas as pd
from tqdm import tqdm
import re
import spacy
from tqdm import tqdm
from unidecode import unidecode
from dateutil import parser
from datetime import datetime
import collections

nlp = spacy.load('en')

In [2]:
def name_disambiguation(name):
    if (name == "Khaleda" or name == "Zia" or name == "Begum Khaleda Zia"):
        return "Khaleda Zia"
    if (name == "Hasina" or name == "Sheikh"):
        return "Sheikh Hasina"
    if (name == "Fakhrul"):
        return "Mirza Fakhrul Islam Alamgir"
    if (name == "Muhith" or name == "AMA Muhith" or name == "MA Muhith"):
        return "Abul Maal Abdul Muhith"
    if (name == "Nizami" or name == "Motiur Rahman"):
        return "Motiur Rahman Nizami"
    if (name == "Modi"):
        return "Narendra Modi"
    if (name == "Bangabandhu" or name == "Sheikh Mujib" or name == "Sheikh Mujib" or name == "Bangabandhu Sheikh Mujibur" or name == "Sheikh Mujibur Rahman"):
        return "Bangabandhu Sheikh Mujibur Rahman"
    if (name == "Tarique"):
        return "Tarique Rahman"
    if (name == "Avijit"):
        return "Avijit Roy"
    if (name == "Mozena"):
        return "Dan Mozena"
    if (name == "Yunus" or name == "Mohammad Yunus"):
        return "Muhammad Yunus"

### Date limit: 2013-07-08 to 2016-06-14 on DT, New Age and Daily Sun

In [3]:
dt = pd.read_json('Data/DT/bd_news_dt.json')
dstar = pd.read_json('Data/DS/news_db.json', lines=True)
dsun = pd.read_pickle('Data/Daily Sun/DailySun_ent_1.pkl')

In [4]:
dstar.head()

,_id,about_reporter,bottom_tagline,breadcrumb,category,content,date_crawled,date_published,generated_keywords,generated_summary,...,ner_unique_person,ner_unique_time,news_url,newspaper,reporter,section,sentiment,shoulder,title,top_tagline
0,{'$oid': '58afca955388c123849ed1be'},NaN,None,[],NaN,People's hopes for the best and confidence in ...,{'$date': '2017-02-24T00:54:28.809Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[parliamentary, fiscal, great, unfulfilled, ca...",This reminded the people of past military regi...,...,"[Quadri, Hasina, Bangla Bhai, Ershad, Ayub Qua...",[],http://www.thedailystar.net/news-detail-17307,The Daily Star,None,Front Page,NaN,NaN,Great expectations unfulfilled,None
1,{'$oid': '58afca985388c123849ed1bf'},NaN,None,[],NaN,"(L-R) Nasir Uddin, Abbas AliArrested Abbas Ali...",{'$date': '2017-02-24T00:54:31.997Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[stones, rab, nasir, museum, took, abbas, offi...",Abbas and Nasir said they stole the statuettes...,...,"[Mohammad Ali, Hamid, Nikhil Chandra Das, Abdu...",[],http://www.thedailystar.net/news-detail-17312,The Daily Star,None,Front Page,NaN,NaN,None,None
2,{'$oid': '58afca9a5388c123849ed1c0'},NaN,None,[],NaN,Daniel Vettori and Brendon McCullum put on rec...,{'$date': '2017-02-24T00:54:34.466Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[zealand, seven, balls, bangladesh, odi, vetto...",The match was over before the scheduled first ...,...,"[Aftab Ahmed, Zaheer, Jacob Oram, Vettori, Far...",[],http://www.thedailystar.net/news-detail-17353,The Daily Star,None,Front Page,NaN,NaN,Kiwis maul Tigers,None
3,{'$oid': '58afca9c5388c123849ed1c1'},NaN,Says BB governor,[],NaN,Bangladesh Bank (BB) Governor Salehuddin Ahmed...,{'$date': '2017-02-24T00:54:36.829Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[wont, outgoing, essentials, soon, governor, i...",Bangladesh Bank in the new year will focus on ...,...,"[Salehuddin Ahmed, Cyclone Sidr]",[],http://www.thedailystar.net/news-detail-17325,The Daily Star,None,Front Page,NaN,NaN,Essentials prices won't go down soon,None
4,{'$oid': '58afca9f5388c123849ed1c2'},NaN,None,[],NaN,Justice KM Sobhan died of cardiac arrest at Bi...,{'$date': '2017-02-24T00:54:39.231Z'},{'$date': '2008-01-01T00:00:00.000Z'},"[passes, sons, sobhan, justice, away, shock, k...","Family sources said Justice Sobhan, also the p...",...,"[KM Sobhan, Ashraful Islam, Sobhan, Zillur Rah...",[],http://www.thedailystar.net/news-detail-17340,The Daily Star,None,Front Page,NaN,NaN,KM Sobhan passes away,None


In [5]:
def conv_to_datetime(date_dict):
    new_datetime = parser.parse(list(date_dict.items())[0][1], ignoretz=True)
    new_datetime = new_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
    return new_datetime

In [6]:
dt['news_publish_date'] = dt['news_publish_date'].apply(conv_to_datetime)

In [7]:
dstar['date_published'] = dstar['date_published'].apply(conv_to_datetime)

In [8]:
dsun['date_published'] = pd.to_datetime(dsun['date_published'])

In [9]:
start_date = datetime(2013, 7, 8)
end_date = datetime(2016, 6, 14)

In [10]:
dt_new = dt.loc[(dt['news_publish_date']>=start_date) & (dt['news_publish_date']<=end_date)]

In [11]:
dstar_new = dstar.loc[(dstar['date_published']>=start_date) & (dstar['date_published']<=end_date)]

In [12]:
dsun_new = dsun.loc[(dsun['date_published']>=start_date) & (dsun['date_published']<=end_date)]

In [13]:
print("dt: {}, dstar: {}, dsun{}".format(dt_new.shape, dstar.shape, dt.shape))

dt: (46611, 17), dstar: (165236, 34), dsun(49055, 17)


In [14]:
dt_new = dt_new.reset_index(drop=True)
dstar_new = dstar_new.reset_index(drop=True)
dsun_new = dsun_new.reset_index(drop=True)

TODO: Need to do a groupby on news_publish_date/date_published and then apply the following function. Diff functions/conditions for dt and dstar/dsun due to how the data is stored

In [15]:
dsun_new.head()

,_id,date_published,image,news_content,news_id,newspaper,reporter,section,url,location_entities,organization_entities,person_entities
0,{'$oid': '5af4592318ce422f881bcaf4'},2013-07-08,"{'src': None, 'caption': None}",ATN BANGLA 19:00 22:00 23:00 Channel i 7:00 ...,123864,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/1238...,[],[],[]
1,{'$oid': '5af4592318ce422f881bcaf5'},2013-07-08,{'src': 'http://www.daily-sun.com/assets/news_...,Solo Art Exhibition By: Biren Shome At: Dhak...,123862,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/1238...,[],[Dhaka Art Centre ],[Dhanmondi]
2,{'$oid': '5af4592418ce422f881bcaf7'},2013-07-08,{'src': 'http://www.daily-sun.com/assets/news_...,NTV 08:45 Bengali Film: Benam Badsha 13:30 ...,123861,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/1238...,"[ Chaabi, 14:00 Saraswatichandra, Caribbean, ...","[Van Helsing, The Three Musketeers, 13:05, NTV...","[21:30 Damages, Veera, 11:30 Veera, Yeh Rishta..."
3,{'$oid': '5af4592418ce422f881bcaf8'},2013-07-08,{'src': 'http://www.daily-sun.com/assets/news_...,RADIO TODAY Islamic Program 12:30 News 1:4...,123863,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/1238...,[],"[ Islamic Program, VOA & RTD]",[Rupali Gaan]
4,{'$oid': '5af4592518ce422f881bcafa'},2013-07-08,{'src': 'http://www.daily-sun.com/assets/news_...,Victoria Beckham has revealed she struggles to...,123874,dailysun,None,/culturetainment,http://www.daily-sun.com/arcprint/details/1238...,"[China, Vogue]",[],"[it\, Spice Girl - who, David Beckham -, Victo..."


In [16]:
# dsun_new.groupby('date_published')['location_entities'].sum().apply(collections.Counter, 1)

In [17]:
dsun_new.groupby('date_published').sum().agg({
    'location_entities': collections.Counter, 
    'organization_entities': collections.Counter,
    'person_entities': collections.Counter
})

,person_entities,location_entities,organization_entities
date_published,,,
2013-07-08,"{'Essex': 2, 'Gaaney Gaaney': 1, 'Giasuddin Se...","{'’We': 1, 'Bardeshi': 2, 'Brahmanbaria': 2, '...","{'Times Of India': 1, 'PGA Tour': 1, 'Manikgan..."
2013-07-09,"{' Proceed': 1, 'Mikhail Myasnikovich': 2, '...","{'’We': 1, 'Dashkahania': 1, 'Upholding': 1, '...","{'The Daily Mail': 1, 'Shaniera Thompson': 1, ..."
2013-07-10,"{'Giasuddin Selim': 1, 'Upazila': 1, 'Nasirudd...","{'USA': 1, 'Taiwan': 1, 'the Padma River': 1, ...","{'BRAC Bank Limited': 1, 'GER/Bayern Munich': ..."
2013-07-11,"{'Wembley': 1, 'Mohamed ElBaradei': 3, 'Warne'...","{'Middlesbrough': 1, 'Taiwan': 1, 'Aleppo': 1,...","{'Times Of India': 4, 'PGA Tour': 1, 'Monwar H..."
2013-07-12,"{'Patna': 1, 'Neha': 1, 'Rakibul Hasan': 2, 'P...","{'Guneet Monga': 1, 'Pyongyang': 2, 'Oregon': ...","{'Times Of India': 1, 'PGA Tour': 1, 'East Way..."
2013-07-13,"{'Muhammad Jobair': 1, 'Parvez Sikdar Panir': ...","{'Tongi': 1, 'Dhaka city': 1, ' Asr 04:45': 1,...","{'Apollo Hospital': 1, 'Safe Food Bill 2013’':..."
2013-07-14,"{'Robert Reich ': 2, 'Zia': 2, 'Denesh Ramdin...","{'Taiwan': 1, 'East Barulia': 2, 'Jiangmen Cit...","{'hawkers’': 1, 'Times Of India': 2, 'KAMPALA'..."
2013-07-15,"{'Essex': 1, 'Gaaney Gaaney': 1, 'Mohamed ElBa...","{'Kamalapur': 1, 'Twitter': 2, 'Paduarbazar': ...","{'Liverpool Arsenal': 1, 'the Police Hall in ..."
2013-07-16,"{'Nababpur Krira Chakra': 1, 'Jacob': 1, 'Wemb...","{'Syria': 4, '14:00 Saraswatichandra': 2, 'Ati...","{'Times Of India': 2, 'Frandy Montrevil': 1, '..."


In [24]:
dstar_new.groupby('date_published').sum().agg({
    'ner_unique_location': collections.Counter, 
    'ner_unique_organization': collections.Counter,
    'ner_unique_person': collections.Counter
})

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


KeyError: 'ner_unique_person'